In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import urllib.request
import time
import re

import pandas as pd
import pprint

from bs4 import BeautifulSoup

In [6]:
# 크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager

# 브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메세지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())

In [7]:
user_agent ="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
chrome_options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(options=chrome_options)

In [9]:
sub_urls = ['high_end_cpus.html', 'mid_range_cpus.html', 'midlow_range_cpus.html', 'low_end_cpus.html']

save_file = False  # 파일 저장 여부
waiting_sec = 2

prd_dict = {
    "name":[],
    "count":[]
    }

for sub in sub_urls:
    url = f'https://www.cpubenchmark.net/{sub}'

    driver.get(url)
    driver.implicitly_wait(10)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    prds = soup.select('#mark > div > div.chart_body > ul > li > a')
    for prd in prds:
        prdname = prd.select_one('a>span.prdname').get_text()
        prdcount = prd.select_one('a>span.count').get_text().replace(',','')
        
        if prdname not in prd_dict['name']:
            prd_dict['name'].append(str(prdname))
            prd_dict['count'].append(int(prdcount))
        else: continue

In [10]:
cpu_df = pd.DataFrame(prd_dict)

In [11]:
cpu_df.head()

,name,count
0,AMD EPYC 9654,126045
1,AMD EPYC 9554P,109858
2,AMD EPYC 9474F,104895
3,Intel Xeon w9-3495X,98285
4,AMD Ryzen Threadripper PRO 5995WX,98215


In [13]:
len(cpu_df)

4417

In [15]:
cpu_df.to_csv("bench_mark_cpu.csv", index=None)